In [38]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv("merch_sales_1.csv")

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7394 entries, 0 to 7393
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Order ID                7394 non-null   int64 
 1   Order Date              7394 non-null   object
 2   Year                    7394 non-null   int64 
 3   Month Name              7394 non-null   object
 4   Product ID              7394 non-null   object
 5   Product Category        7394 non-null   object
 6   Buyer Gender            7394 non-null   object
 7   Buyer Age               7394 non-null   int64 
 8   Order Location          7394 non-null   object
 9   International Shipping  7394 non-null   object
 10  Sales Price             7394 non-null   int64 
 11  Shipping Charges        7394 non-null   int64 
 12  Sales per Unit          7394 non-null   int64 
 13  Quantity                7394 non-null   int64 
 14  Total Sales             7394 non-null   int64 
 15  Rati

In [5]:
df.head()

,Order ID,Order Date,Year,Month Name,Product ID,Product Category,Buyer Gender,Buyer Age,Order Location,International Shipping,Sales Price,Shipping Charges,Sales per Unit,Quantity,Total Sales,Rating,Review
0,189440,21-07-24,2024,July,BF1543,Clothing,Male,30,New Jersey,No,100,0,100,1,100,4,The delivery team handled the product with care.
1,187385,20-07-24,2024,July,BF1543,Clothing,Male,32,Las Vegas,No,100,0,100,1,100,3,Had slight delays but the product was in good ...
2,181844,21-07-24,2024,July,BF1544,Other,Female,26,Cardiff,Yes,9,40,49,1,49,2,Waste of Money.
3,197934,19-08-24,2024,August,BF1544,Other,Male,28,Pittsburgh,No,9,0,9,2,18,3,Had slight delays but the product was in good ...
4,122470,06-01-24,2024,January,BF1545,Other,Female,19,Miami,No,10,0,10,3,30,5,Lack of delivery delays is greatly appreciated.


In [6]:
missing = df.isna().sum()
print(missing)

Order ID                  0
Order Date                0
Year                      0
Month Name                0
Product ID                0
Product Category          0
Buyer Gender              0
Buyer Age                 0
Order Location            0
International Shipping    0
Sales Price               0
Shipping Charges          0
Sales per Unit            0
Quantity                  0
Total Sales               0
Rating                    0
Review                    0
dtype: int64


In [7]:
duplicates = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicates)}")

Number of duplicate rows: 0


In [8]:
print(df.dtypes)

Order ID                   int64
Order Date                object
Year                       int64
Month Name                object
Product ID                object
Product Category          object
Buyer Gender              object
Buyer Age                  int64
Order Location            object
International Shipping    object
Sales Price                int64
Shipping Charges           int64
Sales per Unit             int64
Quantity                   int64
Total Sales                int64
Rating                     int64
Review                    object
dtype: object


In [13]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [14]:
df['Calc_Total'] = df['Sales per Unit'] * df['Quantity']
mismatch = df[df['Total Sales'] != df['Calc_Total']]
print(f"Mismatched Total Sales: {len(mismatch)}")

Mismatched Total Sales: 0


In [15]:
num_cols = ['Sales Price','Shipping Charges','Sales per Unit','Quantity','Total Sales','Rating']
for col in num_cols:
    print(f"{col} negatives or zeros: {(df[col]<0).sum()}")

Sales Price negatives or zeros: 0
Shipping Charges negatives or zeros: 0
Sales per Unit negatives or zeros: 0
Quantity negatives or zeros: 0
Total Sales negatives or zeros: 0
Rating negatives or zeros: 0


In [16]:
print(df.dtypes)

Order ID                           int64
Order Date                datetime64[ns]
Year                               int64
Month Name                        object
Product ID                        object
Product Category                  object
Buyer Gender                      object
Buyer Age                          int64
Order Location                    object
International Shipping            object
Sales Price                        int64
Shipping Charges                   int64
Sales per Unit                     int64
Quantity                           int64
Total Sales                        int64
Rating                             int64
Review                            object
Calc_Total                         int64
dtype: object


In [17]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '', regex=True)

In [18]:
df.columns

Index(['order_id', 'order_date', 'year', 'month_name', 'product_id',
       'product_category', 'buyer_gender', 'buyer_age', 'order_location',
       'international_shipping', 'sales_price', 'shipping_charges',
       'sales_per_unit', 'quantity', 'total_sales', 'rating', 'review',
       'calc_total'],
      dtype='object')

In [24]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

numeric_cols = [
    'sales_price', 'shipping_charges',
    'sales_per_unit', 'quantity',
    'total_sales', 'rating', 'buyer_age'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [25]:
# Перевірка %
missing_pct = df.isna().mean().sort_values(ascending=False)
print(missing_pct)

order_id                  0.0
order_date                0.0
review                    0.0
rating                    0.0
total_sales               0.0
quantity                  0.0
sales_per_unit            0.0
shipping_charges          0.0
sales_price               0.0
international_shipping    0.0
order_location            0.0
buyer_age                 0.0
buyer_gender              0.0
product_category          0.0
product_id                0.0
month_name                0.0
year                      0.0
calc_total                0.0
dtype: float64


In [26]:
# Видаляємо критичні пропуски
df = df.dropna(subset=[
    'order_id', 'order_date',
    'sales_per_unit', 'quantity', 'total_sales'
])

# buyer_age можна заповнити медіаною
df['buyer_age'] = df['buyer_age'].fillna(df['buyer_age'].median())

# rating — якщо пропущений, не видаляємо (може бути без відгуку)

In [27]:
df['calc_total'] = df['sales_per_unit'] * df['quantity']

# Залишаємо тільки коректні
df = df[abs(df['total_sales'] - df['calc_total']) < 0.01]

In [28]:
# Без негативних або нульових продажів
df = df[(df['quantity'] > 0) &
        (df['sales_per_unit'] > 0) &
        (df['total_sales'] > 0)]

# Rating в межах 1–5
df = df[(df['rating'].isna()) |
        ((df['rating'] >= 1) & (df['rating'] <= 5))]

In [29]:
df['international_shipping'] = df['international_shipping'].str.strip().str.lower()

df['international_flag'] = df['international_shipping'].map({
    'yes': 1,
    'no': 0
})

In [33]:
df['proxy_margin'] = df['total_sales'] - df['shipping_charges']
df['margin_ratio'] = df['proxy_margin'] / df['total_sales']

In [34]:
df['shipping_ratio'] = df['shipping_charges'] / df['total_sales']

In [35]:
df['order_size'] = pd.qcut(
    df['total_sales'],
    q=3,
    labels=['small', 'medium', 'large']
)

In [36]:
df['age_group'] = pd.cut(
    df['buyer_age'],
    bins=[0,25,40,60,100],
    labels=['18-25','26-40','41-60','60+']
)

In [40]:
overall_metrics = {
    "total_revenue": float(df['total_sales'].sum()),
    "total_proxy_margin": float(df['proxy_margin'].sum()),
    "avg_margin_ratio": float(df['margin_ratio'].mean()),
    "total_orders": int(df['order_id'].nunique())
}

In [41]:
shipping_analysis = (
    df.groupby('international_flag')
      .agg(
          total_revenue=('total_sales','sum'),
          total_proxy_margin=('proxy_margin','sum'),
          avg_margin_ratio=('margin_ratio','mean'),
          total_orders=('order_id','nunique'),
          avg_shipping_ratio=('shipping_ratio','mean')
      )
      .reset_index()
)

shipping_analysis_dict = shipping_analysis.to_dict(orient='records')

In [42]:
# Кореляція
correlation = float(df[['shipping_ratio','margin_ratio']].corr().iloc[0,1])

# Високі shipping витрати (> 40%)
high_shipping = df[df['shipping_ratio'] > 0.4]

high_shipping_metrics = {
    "orders_count": int(high_shipping['order_id'].nunique()),
    "revenue_share": float(high_shipping['total_sales'].sum() / df['total_sales'].sum()),
    "avg_margin_ratio": float(high_shipping['margin_ratio'].mean())
}

In [43]:
avg_margin = df['margin_ratio'].mean()

false_growth = df[
    (df['total_sales'] > df['total_sales'].median()) &
    (df['margin_ratio'] < avg_margin)
]

false_growth_metrics = {
    "orders_count": int(false_growth['order_id'].nunique()),
    "revenue_share": float(false_growth['total_sales'].sum() / df['total_sales'].sum()),
    "avg_margin_ratio": float(false_growth['margin_ratio'].mean())
}

In [45]:
report_data = {
    "business_case": "International Shipping vs Profitability",
    "overall_metrics": overall_metrics,
    "shipping_comparison": shipping_analysis_dict,
    "shipping_margin_correlation": correlation,
    "high_shipping_segment": high_shipping_metrics,
    "false_growth_segment": false_growth_metrics
}

# Збереження у файл
with open("retail_ai_report_data.json", "w") as f:
    json.dump(report_data, f, indent=4)

In [60]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

<class 'ModuleNotFoundError'>: No module named 'dotenv'

In [61]:
load_dotenv()

<class 'NameError'>: name 'load_dotenv' is not defined

In [62]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

<class 'NameError'>: name 'os' is not defined

In [63]:
PROMPT = """
# AI Business Report Generation Prompt

## Business Context
You are a senior business analyst. The client is a retail company selling products locally and internationally. The company observes growth in total sales but is uncertain about the profitability of international orders due to high shipping charges. The client wants a management-ready report focused on **International Shipping vs Profitability**.

## Input Data
You are provided a structured JSON file containing:

- Overall metrics (total revenue, total proxy margin, average margin ratio, total orders)
- Shipping comparison (International vs Local: revenue, proxy margin, margin ratio, orders, avg shipping ratio)
- Shipping-margin correlation
- High shipping segment metrics (orders with shipping ratio > 0.4)
- False growth segment metrics (high revenue orders with below-average margin)

## Task
1. Analyze the JSON data and extract **actionable insights** for management.
2. Identify **key profitability drivers** and risks related to international shipping.
3. Highlight **segments where high sales do not translate to high profit**.
4. Provide **3–5 concise insights** that can be directly communicated to executives.
5. Suggest **1–2 concrete business decisions** or scenarios to improve profitability (e.g., shipping policy adjustments, pricing strategy, order threshold).
6. Present all results in **clear, structured Markdown** suitable for management review.

## Output Structure (Markdown)
- **Executive Summary**: 3–5 key insights + one actionable recommendation.
- **Key Metrics**: short table highlighting revenue, margin, margin ratio, orders.
- **Analysis of International Shipping**: commentary on profitability, margin leakage, high shipping costs.
- **False Growth Segment**: highlight orders contributing to revenue but eroding profit.
- **Decision Scenarios**: concise actionable options for management.
- **Optional Appendix**: include relevant numbers from JSON.

## Style & Tone
- Professional, concise, C-level audience.
- Avoid technical jargon; focus on business impact.
- Use clear headings, bullet points, and tables where appropriate.
"""


In [65]:
report_json_str = json.dumps(report_data, indent=4)

prompt = f"""
{PROMPT}

INPUT DATA:
{report_json_str}
"""

In [66]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert business analyst."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

<class 'NameError'>: name 'client' is not defined

In [67]:
business_report_md = response.choices[0].message.content

<class 'NameError'>: name 'response' is not defined

In [68]:
with open("business_report.md", "w", encoding="utf-8") as f:
    f.write(business_report_md)

<class 'NameError'>: name 'business_report_md' is not defined